In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("small_movie_reviews.csv")

/tmp/ipykernel_1253/3904227517.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("small_movie_reviews.csv")


In [3]:
df.head()

,Unnamed: 0,rating,review_title,text,images_x,asin,parent_asin,user_id,timestamp,helpful_vote,...,features,description,price,images_y,videos,store,categories,details,bought_together,author
0,0,5.0,Five Stars,"Amazon, please buy the show! I'm hooked!",[],B013488XFS,B013488XFS,AGGZ357AO26RQZVRLGU4D4N52DZQ,1440385637000,0,...,"['IMDb 8.1', '2017', '10 episodes', 'X-Ray', '...",['A\xa0con man (Giovanni Ribisi) on the run fr...,NaN,[{'360w': 'https://images-na.ssl-images-amazon...,[],NaN,"['Suspense', 'Drama']","{'Content advisory': ['Nudity', 'violence', 's...",NaN,NaN
1,10,5.0,Five Stars,LOVE this movie! Hits you in the feels!,[],B01LWY8995,B01LWY8995,AGXVBIUFLFGMVLATYXHJYL4A5Q7Q,1485870227000,0,...,NaN,NaN,NaN,[{'360w': 'https://images-na.ssl-images-amazon...,[],NaN,NaN,"{'Content advisory': ['Violence', 'alcohol use...",NaN,NaN
2,20,5.0,Beautiful Move for the Entire Familia!,We loved Coco! My husband (Mexican) and I (Cau...,[],B0779KMWG1,B0779KMWG1,AHTTU2FL6FCNBBAESCJHOHHSSW7A,1513539059497,7,...,NaN,NaN,NaN,[{'360w': 'https://images-na.ssl-images-amazon...,[],NaN,NaN,{'Audio languages': ['English Dialogue Boost: ...,NaN,NaN
3,30,5.0,Five Stars,VERY GOOD MOVIE: WAS LIKED BY ALL WHO SEEN IT,[],B001QTXM5Y,B001QTXM5Y,AEFKF6R2GUSK2AWPSWRR4ZO36JVQ,1414899539000,0,...,[],"['Product Description', 'Watchmen: Special Edi...",12.40,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Watchmen home-video trailer', 'url...","Malin Akerman (Actor), Billy Crudup (Act...","['Movies & TV', 'Genre for Featured Categories...","{'Genre': 'Science Fiction & Fantasy, Drama, A...",NaN,NaN
4,40,5.0,FIFTH ELEMENT ultimate edition,one of my classics: This one I and friends can...,[],B0006GVJE4,B0006GVJE4,AEFKF6R2GUSK2AWPSWRR4ZO36JVQ,1364088328000,0,...,[],"['Product Description', ""New York cab driver K...",19.12,[{'thumb': 'https://m.media-amazon.com/images/...,[],"Bruce Willis (Actor), Gary Oldman (Actor...","['Movies & TV', 'Science Fiction & Fantasy', '...","{'Aspect Ratio': '2.35:1', 'Is Discontinued By...",NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         100000 non-null  int64  
 1   rating             100000 non-null  float64
 2   review_title       99990 non-null   object 
 3   text               99993 non-null   object 
 4   images_x           100000 non-null  object 
 5   asin               100000 non-null  object 
 6   parent_asin        100000 non-null  object 
 7   user_id            100000 non-null  object 
 8   timestamp          100000 non-null  int64  
 9   helpful_vote       100000 non-null  int64  
 10  verified_purchase  100000 non-null  bool   
 11  main_category      99082 non-null   object 
 12  movie_title        45657 non-null   object 
 13  subtitle           1997 non-null    object 
 14  average_rating     99999 non-null   float64
 15  rating_number      99999 non-null   float64
 16  fea

In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import LatentDirichletAllocation
import gensim
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

def preprocess(corpus):
    """
    This function takes in a pandas.Series() of a corpus of text data as an argument.
    This function should output an indexed vocabulary and preprocessed tokens.

    Input:
        corpus (pandas.Series): a series of text data
    Output:
        vocab (dict): a dictionary of indexed vocabulary
        tokens (list): a list of preprocessed tokens
    """
    # Initialize empty lists to store tokens and vocabulary
    vocab = {}
    tokens = []

    # Iterate over each document in the corpus
    for i, doc in enumerate(corpus):
        # Tokenize the document
        tokenized_doc = word_tokenize(doc)

        # Convert tokens to lowercase
        tokenized_doc = [token.lower() for token in tokenized_doc]

        # Filter out punctuation and non-alphanumeric characters
        tokenized_doc = [token for token in tokenized_doc if token.isalnum()]

        # Remove duplicate tokens from the document
        tokenized_doc = list(set(tokenized_doc))

        # Add document tokens to the tokens list
        tokens.append(tokenized_doc)

        # Update the vocabulary
        for token in tokenized_doc:
            if token not in vocab:
                vocab[token] = len(vocab)

    return vocab, tokens

def encode(preprocessed_tokens, encoding_method):
    """
    This function takes in two arguments: 1) the preprocessed token outputs of the preprocess() function,
    and 2) the desired encoding method.
    
    It then encodes the preprocessed tokens using the specified encoding method and returns the encoded tokens.
    
    The available encoding methods are 'Bag-of-Words', 'TF-IDF', and 'Word2Vec'.
    """
    if encoding_method == 'Bag-of-Words':
        # Initialize a CountVectorizer model
        vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')

        # Join the tokens of each document into a single string
        preprocessed_documents = [' '.join(doc_tokens) for doc_tokens in preprocessed_tokens]

        # Encode the preprocessed tokens using the vectorizer
        encoding = vectorizer.fit_transform(preprocessed_documents)
    elif encoding_method == 'TF-IDF':
        # Initialize a TF-IDF vectorizer
        vectorizer = TfidfVectorizer(token_pattern=r'\b\w+\b')

        # Join the tokens of each document into a single string
        preprocessed_documents = [' '.join(doc_tokens) for doc_tokens in preprocessed_tokens]

        # Encode the preprocessed tokens using the vectorizer
        encoding = vectorizer.fit_transform(preprocessed_documents)
    elif encoding_method == 'Word2Vec':
        # Initialize a Word2Vec model
        model = gensim.models.Word2Vec(preprocessed_tokens, min_count=1)

        # Encode the preprocessed tokens using the model
        encoding = []
        for doc_tokens in preprocessed_tokens:
            doc_vec = np.zeros(model.vector_size)
            for token in doc_tokens:
                if token in model.wv:
                    doc_vec += model.wv[token]
            encoding.append(doc_vec)
        encoding = np.array(encoding)
    else:
        raise ValueError("Invalid encoding method. Choose from 'Bag-of-Words', 'TF-IDF', or 'Word2Vec'.")

    return encoding

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
df["text"] = df["text"].fillna("")

In [7]:


# Preprocess the corpus
vocab, tokens = preprocess(df["text"])

# Encode the preprocessed tokens using the specified encoding method
encoding_method = 'Bag-of-Words'
encoded_tokens = encode(tokens, encoding_method)

In [8]:
encoded_tokens

<100000x65815 sparse matrix of type '<class 'numpy.int64'>'
	with 3130010 stored elements in Compressed Sparse Row format>

In [9]:
encoded_tokens[0][0][0]

<1x65815 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [10]:
encoding_method = 'TF-IDF'
encoded_tokens = encode(tokens, encoding_method)

In [11]:
encoded_tokens

<100000x65815 sparse matrix of type '<class 'numpy.float64'>'
	with 3130010 stored elements in Compressed Sparse Row format>

In [12]:
encoded_tokens[0][0][0]

<1x65815 sparse matrix of type '<class 'numpy.float64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [13]:
encoding_method = 'Word2Vec'
encoded_tokens = encode(tokens, encoding_method)

In [14]:
encoded_tokens

array([[  5.83584717,  -1.20755839,   1.07774064, ...,  -1.42397977,
         -5.565654  ,  -1.90248314],
       [  4.53061621,   5.41090962,   0.04808341, ...,  -3.84772317,
         -2.34061489,   1.1168531 ],
       [ 31.77329556,  27.85414376,   5.99537518, ..., -35.7595458 ,
         -0.83008845,  17.62207972],
       ...,
       [  2.67567014,  14.53567044,  15.29970875, ..., -42.17137759,
        -25.03599469,   4.16676166],
       [ -0.28432921,   8.53320832,   3.76346451, ...,  -9.86925241,
          4.47263063,   0.19936887],
       [ 10.2640025 ,   2.47418444,   3.05794685, ...,  -1.54698901,
         -5.68287043,   0.81294036]])